In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import random

mod = "distilbert/distilgpt2"


tokenizer = AutoTokenizer.from_pretrained(mod)
model = AutoModelForCausalLM.from_pretrained(mod)
model.eval()


/Users/gusfridell/battle/b-vae/test-train/torch-env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/gusfridell/battle/b-vae/test-train/torch-env/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/gusfridell/battle/b-vae/test-train/torch-env/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <ABE0EE74-6D97-3B8C-B690-C44754774FBC> /Users/gusfridell/battle/b-vae/test-train/torch-env/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <CDAC6E34-8608-3E70-8B2F-32BCD38E90FB> /Users/gusfridell/battle/b-vae/test-train/torch-env/lib/python3.8/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvi

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
# Choose your paragraph
paragraph = "The trousers were sent to you (1) as a means of testing your initiative (A clever, wide-awake business concern should be able to make three-quarter length trousers a by-word of masculine fashion. Your advertising and merchandising programs are obviously faulty.) and (2) as a means of testing your ability to meet the standards requisite in a distributor of our quality product. (Our loyal and dependable outlets can vend any trouser bearing the Levy label no matter how abominable their design and construction. You are apparently a faithless people.) We do not wish to be bothered in the future by such tedious complaints. Please confine your correspondence to orders only. We are a busy and dynamic organization whose mission needless effrontery and harassment can only hinder. If you molest us again, sir, you may feel the sting of the lash across your pitiful shoulders."
inputs = tokenizer(paragraph, return_tensors="pt")
with torch.no_grad():
    outputs = model(inputs.input_ids, labels=inputs.input_ids)
    log_likelihood = outputs.loss * inputs.input_ids.shape[1]
    perplexity = torch.exp(log_likelihood / inputs.input_ids.shape[1])

words = paragraph.split()
random.shuffle(words)
shuffled_paragraph = " ".join(words)

shuffled_inputs = tokenizer(shuffled_paragraph, return_tensors="pt")
with torch.no_grad():
    shuffled_outputs = model(shuffled_inputs.input_ids, labels=shuffled_inputs.input_ids)
    shuffled_log_likelihood = shuffled_outputs.loss * shuffled_inputs.input_ids.shape[1]
    shuffled_perplexity = torch.exp(shuffled_log_likelihood / shuffled_inputs.input_ids.shape[1])



tensor(109.6611)

In [3]:
print(shuffled_perplexity)
print(perplexity)

tensor(1779.8390)
tensor(109.6611)


In [7]:
prompt = "Once upon a time"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
max_length = len(input_ids[0]) + 50

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id
if torch.cuda.is_available():
    model = model.to('cuda')
    input_ids = input_ids.to('cuda')
greedy_output = model.generate(
    input_ids,
    max_length=max_length,
    do_sample=False,
    pad_token_id=tokenizer.eos_token_id
)

print(tokenizer.decode(greedy_output[0]))

temperatures = [0.3, 0.6, 0.9, 1.2, 1.5]
for T in temperatures:
    print(f"temp {T}")
    sample_output = model.generate(
        input_ids,
        max_length=max_length,
        do_sample=True,
        temperature=T,
        pad_token_id=tokenizer.eos_token_id
    )
    print(tokenizer.decode(sample_output[0]))

Once upon a time of war, the United States was the only country in the world to have a military presence. The United States was the only country in the world to have a military presence. The United States was the only country in the world to have a military presence
temp 0.3
Once upon a time when the world was in a state of flux, the world was in a state of flux. The world was in a state of flux. The world was in a state of flux. The world was in a state of flux. The world was in
temp 0.6
Once upon a time when the people of the world are getting in the way of making a living, I think we all need to be prepared for the inevitable.


I think our current climate is very different than that of the past.
The United States has
temp 0.9
Once upon a time of war and violence, it would be an honour to have the opportunity to do so. The French will always have the right to use the opportunity, and if it is not offered, the decision made over the course of the war might also change the
temp 1.2
On